# AMBRS × CAMP demo (H₂SO₄ + SOAG condensation) 

This notebook mirrors the AMBRS PPE flow and runs **baseline vs CAMP** for both MAM4 and PartMC.

**Highlights**
- CAMP files are written per-scenario under `<run_dir>/<scenario>/camp/<model>/`.
- Models are pointed at an **absolute** `camp_files.json`.
- Runner passes a dynamic-library env so `libcamp` is found (macOS/Linux).
- The CAMP mechanism is **general**; we include H₂SO₄→SO₄ and SOAG→SOA via `SIMPOL_PHASE_TRANSFER` as the first example.


In [1]:
# --- Paths & run controls ---
from pathlib import Path
import os

MAM4_EXE = os.environ.get('MAM4_EXE', 'mam4')
PARTMC_EXE = os.environ.get('PARTMC_EXE', 'partmc')

ROOT = Path('runs_camp_demo').resolve()
ROOT.mkdir(parents=True, exist_ok=True)

N_SCEN = 3      # small PPE for demo
DT = 300.0      # s
NSTEPS = 12     # total time = DT*NSTEPS

BASE = 'baseline'
CAMP = 'camp'

print('Run root:', ROOT)

Run root: /Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs_camp_demo


In [2]:
# --- AMBRS imports ---
import ambrs
from ambrs import mam4, partmc
from ambrs.runners import PoolRunner
from ambrs.camp import CampConfig
from ambrs.aerosol import AerosolSpecies, AerosolModeDistribution, AerosolModalSizeDistribution, AerosolProcesses
from ambrs.gas import GasSpecies
try:
    from ambrs.ppe import EnsembleSpecification
except Exception:
    EnsembleSpecification = None


## CAMP mechanism (general; H₂SO₄→SO₄ and SOAG→SOA via SIMPOL)

In [3]:
# Prefer to give CampConfig a conda lib/ dir so dyld/ld can find libcamp
camp = CampConfig(
    mechanism_name='ambrs_h2so4_soag',
    explicit_lib_dirs=(Path('/Users/fier887/Downloads/ambuilder/build/lib/libcamp.1.1.0.dylib'),
    #(Path(os.environ.get('CONDA_PREFIX',''))/'lib',
    )
)



## Build the PPE using AMBRS tools (compatible with current demos)

In [4]:
# # Gas species (include H2SO4 and SOAG so CAMP has sources)
# # fixme: SOAG molar mass not right
# gases = [GasSpecies('H2SO4', molar_mass=98.), GasSpecies('SO2',molar_mass=64.)]#, GasSpecies('SOAG', molar_mass=200.)]

# # Aerosol species (include SO4 & SOA for CAMP condensation products)
# aerosols = [
#     AerosolSpecies('SO4', density=1770.0, ions_in_soln=2, molar_mass=115.10734, hygroscopicity=0.0),
#     AerosolSpecies('OC',  density=1000.0, ions_in_soln=0, molar_mass=12.011,    hygroscopicity=0.14),
#     AerosolSpecies('ARO1', density=1000.0, ions_in_soln=0, molar_mass=150.0,     hygroscopicity=0.14),
#     AerosolSpecies('SOA', density=1000.0, ions_in_soln=0, molar_mass=150.0,     hygroscopicity=0.14),
#     AerosolSpecies('BC',  density=1500.0, ions_in_soln=0, molar_mass=12.011,    hygroscopicity=0.0),
#     AerosolSpecies('OIN', density=2650.0, ions_in_soln=0, molar_mass=60.0843,   hygroscopicity=0.0),
#     AerosolSpecies('Na',  density=2160.0, ions_in_soln=1, molar_mass=22.9898,   hygroscopicity=0.0),
# ]

# # 4 modes; PPE will fill numbers/GMD/GSD based on your spec
# modes = [
#     AerosolModeDistribution('accumulation', species=aerosols),
#     AerosolModeDistribution('aitken',       species=aerosols),
#     AerosolModeDistribution('coarse',       species=aerosols),
#     AerosolModeDistribution('primary carbon', species=aerosols),
# ]
# modal = AerosolModalSizeDistribution(modes=modes)

# # Build a specification in the style used by current demos
# if EnsembleSpecification is None:
#     raise RuntimeError('Could not import EnsembleSpecification from ambrs.ppe; please pull the latest integration branch.')
# spec = EnsembleSpecification(
#     gases=gases,
#     aerosol_species=aerosols,
#     modal_size=modal,
#     temperature=(290.0, 290.0),
#     pressure=(101325.0, 101325.0),
#     relative_humidity=(0.5, 0.5),
# )

# # Try several creation APIs to be robust to minor branch differences
# ensemble = None
# err = None
# for make in (
#     lambda: ambrs.lhs(specification=spec, n=N_SCEN),
#     lambda: ambrs.ppe.lhs(specification=spec, n=N_SCEN),
# ):
#     try:
#         ensemble = make()
#         break
#     except Exception as e:
#         err = e
# if ensemble is None:
#     raise RuntimeError(f'Failed to create ensemble via lhs; last error: {err}')
# print(f'Built ensemble of {len(ensemble.scenarios)} scenarios')


In [ ]:
# -----------------------------------------------------------
# Import modules
# -----------------------------------------------------------
import os
import logging
from math import log10
from pathlib import Path

import numpy as np
import scipy.stats as stats

import ambrs
from pathlib import Path

# -----------------------------------------------------------
# Configure simualtions
# -----------------------------------------------------------

n = 5        # number of ensemble members
n_part = 2000  # PartMC particles per run

dt = 60      # timestep [s]
nstep = 60   # 60 min run

p0 = 101325  # reference pressure [Pa]
h0 = 500     # reference height [m]

# -----------------------------------------------------------
# Make directories for simulation output
# -----------------------------------------------------------
ensemble_name = str(int(dt*nstep/60)) + "min_" + str(n) + "particles"
repo_root = Path.cwd().resolve()
root_dir =  repo_root / "runs" 
ensemble_dir =  root_dir / ensemble_name
partmc_dir = os.path.join(ensemble_dir, "partmc_runs")
mam4_dir = os.path.join(ensemble_dir, "mam4_runs")

for d in [root_dir, ensemble_dir, partmc_dir, mam4_dir]:
    os.makedirs(d, exist_ok=True)


# -----------------------------------------------------------
# Define species
# -----------------------------------------------------------
so4 = ambrs.AerosolSpecies("SO4", molar_mass=96., density=1800., hygroscopicity=0.65)
pom = ambrs.AerosolSpecies("OC", molar_mass=12.01, density=1000., hygroscopicity=0.001)
soa = ambrs.AerosolSpecies("MSA", molar_mass=40., density=2600., hygroscopicity=0.53)
bc  = ambrs.AerosolSpecies("BC",  molar_mass=12.01, density=1800., hygroscopicity=0.)
dst = ambrs.AerosolSpecies("OIN", molar_mass=135.065, density=2600., hygroscopicity=0.1)
na  = ambrs.AerosolSpecies("Na",  molar_mass=23., density=2200., hygroscopicity=0.53)
cl  = ambrs.AerosolSpecies("Cl",  molar_mass=35.5, density=2200., hygroscopicity=0.53)
ncl = na
h2o = ambrs.AerosolSpecies("H2O", molar_mass=18., density=1000., ions_in_soln=1)

so2   = ambrs.GasSpecies("SO2", molar_mass=64.07)
h2so4 = ambrs.GasSpecies("H2SO4", molar_mass=98.079)

# -----------------------------------------------------------
# Define aerosol processes
# -----------------------------------------------------------
processes = ambrs.AerosolProcesses(
    coagulation=True,
    condensation=True,
)

# -----------------------------------------------------------
# Ensemble specification
# -----------------------------------------------------------
spec = ambrs.EnsembleSpecification(
    name=ensemble_name,
    aerosols=(so4, pom, soa, bc, dst, ncl, h2o),
    gases=(so2, h2so4),
    size=ambrs.AerosolModalSizeDistribution(modes=[
        ambrs.AerosolModeDistribution(
            name="accumulation",
            species=[so4, pom, soa, bc, dst, ncl],
            number=stats.uniform(1e7, 1e10),
            geom_mean_diam=stats.rv_discrete(values=([1.1e-7], [1.])),
            log10_geom_std_dev=log10(1.6),
            mass_fractions=[
                stats.rv_discrete(values=([1.], [1.])),  # SO4
                stats.rv_discrete(values=([0.], [1.])),  # POM
                stats.rv_discrete(values=([0.], [1.])),  # SOA
                stats.rv_discrete(values=([0.], [1.])),  # BC
                stats.rv_discrete(values=([0.], [1.])),  # DST
                stats.rv_discrete(values=([0.], [1.])),  # NCL
            ],
        ),
        ambrs.AerosolModeDistribution(
            name="aitken",
            species=[so4, soa, ncl],
            number=stats.uniform(1e7, 1e11),
            geom_mean_diam=stats.rv_discrete(values=([2.6e-8], [1.])),
            log10_geom_std_dev=log10(1.6),
            mass_fractions=[
                stats.rv_discrete(values=([1.], [1.])),  # SO4
                stats.rv_discrete(values=([0.], [1.])),  # SOA
                stats.rv_discrete(values=([0.], [1.])),  # NCL
            ],
        ),
        ambrs.AerosolModeDistribution(
            name="coarse",
            species=[dst, ncl, so4, bc, pom, soa],
            number=stats.uniform(1e6, 1e7),
            geom_mean_diam=stats.rv_discrete(values=([2e-6], [1.])),
            log10_geom_std_dev=log10(1.8),
            mass_fractions=[
                stats.rv_discrete(values=([0.], [1.])),
                stats.rv_discrete(values=([0.], [1.])),
                stats.rv_discrete(values=([1.], [1.])),  # SO4
                stats.rv_discrete(values=([0.], [1.])),
                stats.rv_discrete(values=([0.], [1.])),
                stats.rv_discrete(values=([0.], [1.])),
            ],
        ),
        ambrs.AerosolModeDistribution(
            name="primary carbon",
            species=[pom, bc],
            number=stats.rv_discrete(values=([0.], [1.])),
            geom_mean_diam=stats.loguniform(1e-8, 5e-8),
            log10_geom_std_dev=log10(1.8),
            mass_fractions=[
                stats.rv_discrete(values=([1.], [1.])),  # POM
                stats.rv_discrete(values=([0.], [1.])),  # BC
            ],
        ),
    ]),
    gas_concs=tuple([stats.rv_discrete(values=([0.], [1.])), stats.uniform(1e-10, 1e-8)]),
    #gas_concs=tuple([stats.uniform(1e-10, 1e-8) for _ in range(2)]),
    flux=stats.uniform(1e-11, 1e-8), # not doing anything
    relative_humidity=stats.rv_discrete(values=([0.5], [1.])),
    #relative_humidity=stats.uniform(0, 0.99),
    temperature=stats.uniform(240, 70),
    pressure=p0,
    height=h0,
)

ensemble = ambrs.lhs(specification=spec, n=n)
scenario_names = [str(ii).zfill(1) for ii in range(1, len(ensemble.flux)+1)]


print(mam4_dir)
# -----------------------------------------------------------
# Run MAM4
# -----------------------------------------------------------
mam4 = ambrs.mam4.AerosolModel(processes=processes, camp=camp)
mam4_inputs = mam4.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)
mam4_runner = ambrs.PoolRunner(
    model=mam4,
    executable="mam4",
    root=mam4_dir,
    num_processes=1,
)
mam4_runner.run(mam4_inputs)

# -----------------------------------------------------------
# Run PartMC
# -----------------------------------------------------------
partmc = ambrs.partmc.AerosolModel(
    processes=processes,
    run_type="particle",
    n_part=n_part,
    n_repeat=1,
    camp=camp
)
partmc_inputs = partmc.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)
partmc_runner = ambrs.PoolRunner(
    model=partmc,
    executable="partmc",
    root=partmc_dir,
    num_processes=1,
)
partmc_runner.run(partmc_inputs)


mam4: one or more existing scenario directories found. Overwriting contents...


/Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/60min_5particles/mam4_runs


FileNotFoundError: [Errno 2] No such file or directory: 'mam4'

## Run MAM4 — baseline vs CAMP

In [ ]:
# mam_base = mam4.AerosolModel(
#     processes=AerosolProcesses(gas_phase_chemistry=True, condensation=True, coagulation=True, nucleation=False)
# )
# mam_camp = mam4.AerosolModel(
#     processes=AerosolProcesses(gas_phase_chemistry=True, condensation=False, coagulation=True, nucleation=False),
#     camp=camp,
# )

# mam_root = ROOT/'mam4'
# (mam_root/BASE).mkdir(parents=True, exist_ok=True)
# (mam_root/CAMP).mkdir(parents=True, exist_ok=True)

# mam_inputs_base = mam_base.create_inputs(ensemble=ensemble, dt=DT, nstep=NSTEPS)
# mam_inputs_camp = mam_camp.create_inputs(ensemble=ensemble, dt=DT, nstep=NSTEPS)

# PoolRunner(mam_base, MAM4_EXE, str(mam_root/BASE), num_processes=1).run(mam_inputs_base)
# PoolRunner(mam_camp, MAM4_EXE, str(mam_root/CAMP), num_processes=1).run(mam_inputs_camp)
# print('MAM4 runs finished')


AttributeError: 'AerosolModel' object has no attribute 'AerosolModel'

## Run PartMC — baseline vs CAMP

In [ ]:
# pm_base = partmc.AerosolModel(
#     processes=AerosolProcesses(gas_phase_chemistry=False, condensation=True, coagulation=True, nucleation=False),
#     run_type='particle', n_part=20000, n_repeat=1
# )
# pm_camp = partmc.AerosolModel(
#     processes=AerosolProcesses(gas_phase_chemistry=False, condensation=False, coagulation=True, nucleation=False),
#     run_type='particle', n_part=20000, n_repeat=1, camp=camp
# )

# pm_root = ROOT/'partmc'
# (pm_root/BASE).mkdir(parents=True, exist_ok=True)
# (pm_root/CAMP).mkdir(parents=True, exist_ok=True)

# pm_inputs_base = pm_base.create_inputs(ensemble=ensemble, dt=DT, nstep=NSTEPS)
# pm_inputs_camp = pm_camp.create_inputs(ensemble=ensemble, dt=DT, nstep=NSTEPS)

# PoolRunner(pm_base, PARTMC_EXE, str(pm_root/BASE), num_processes=1).run(pm_inputs_base)
# PoolRunner(pm_camp, PARTMC_EXE, str(pm_root/CAMP), num_processes=1).run(pm_inputs_camp)
# print('PartMC runs finished')


## Diagnostics — parity and conservation checks

In [ ]:
import numpy as np
from netCDF4 import Dataset
from pathlib import Path

def read_series_mam4(run_dir):
    out = {}
    nc = Dataset(Path(run_dir)/'mam_output.nc')
    def tryv(names):
        for nm in names:
            if nm in nc.variables: return np.array(nc.variables[nm][:])
        return None
    out['H2SO4_g'] = tryv(['h2so4_gas','H2SO4_gas','H2SO4'])
    out['SO4_a']   = tryv(['so4_aero','SO4_aero','SO4'])
    return out

def read_series_partmc(scen_dir):
    out = {'H2SO4_g': None, 'SO4_a': None}
    outs = sorted((Path(scen_dir)/'out').glob('*.nc'))
    if not outs: return out
    nc = Dataset(outs[-1])
    names = []
    if 'gas_species' in nc.variables:
        names = nc.variables['gas_species'].names.split(',')
    if 'gas_mixing_ratio' in nc.variables and 'H2SO4' in names:
        vals = np.array(nc.variables['gas_mixing_ratio'])
        out['H2SO4_g'] = vals[:, names.index('H2SO4')]
    return out

def mean_rel_err(a, b):
    if a is None or b is None: return float('nan')
    denom = np.maximum(np.abs(a), 1e-30)
    return float(np.mean(np.abs(a-b)/denom))

def total_S_proxy(series):
    # simple proxy: H2SO4_g + SO4_a (units must be consistent inside each model)
    a = series.get('H2SO4_g')
    b = series.get('SO4_a')
    if a is None or b is None: return None
    L = min(len(a), len(b))
    return a[:L] + b[:L]

def run_checks(model_root):
    errs = []
    scen_dirs_base = sorted((model_root/BASE).glob('*/'))
    scen_dirs_camp = sorted((model_root/CAMP).glob('*/'))
    for base_dir, camp_dir in zip(scen_dirs_base, scen_dirs_camp):
        if (model_root.name) == 'mam4':
            base = read_series_mam4(base_dir)
            camp = read_series_mam4(camp_dir)
        else:
            base = read_series_partmc(base_dir)
            camp = read_series_partmc(camp_dir)
        errs.append({
            'H2SO4_rel_err': mean_rel_err(base['H2SO4_g'], camp['H2SO4_g']),
            'SO4_rel_err'  : mean_rel_err(base['SO4_a'],   camp['SO4_a']),
            'total_S_span_base': (None if total_S_proxy(base) is None else float(np.ptp(total_S_proxy(base)))),
            'total_S_span_camp': (None if total_S_proxy(camp) is None else float(np.ptp(total_S_proxy(camp)))),
        })
    return errs

mam_errs = run_checks(ROOT/'mam4')
pm_errs  = run_checks(ROOT/'partmc')
print('MAM4 checks:', mam_errs)
print('PartMC checks:', pm_errs)


MAM4 checks: []
PartMC checks: []
